In [24]:
import numpy as np
import Parser
instance = Parser.TSPInstance('data/data_101.tsp')
instance.readData()
data = np.copy(instance.data)

## Helper methods

In [25]:
def norm_random():
    return np.random.randint(100)/100

This function represents the amount of attraciveness of a path against the pheromones put in it 
$({\tau^{\alpha}_x}_y)\times({\zeta^{\beta}_x}_y)$


In [26]:
def attract_pherm(x,y,alpha,beta,pheromone,data):
    return (pheromone[x][y]**alpha)*((1/data[x][y])**beta)

In [27]:
def show_paths(ants):
    print('completed iteration : ')
    i = 1
    for ant in ants :
        print('ant',i,':',ant.path,', cost :',ant.cost)
        i+=1

## Class of ant 

In [28]:
class Ant:
    def __init__(self,begin):
        self.path = [begin]
        self.cost = 0.0
    
    #finding next node to visit in stochastic way
    def next_node(self,data=[],pheromone=[],alpha=0.5,beta=0.5,random_rate=0.3):
        next_node = -1
        #case of an ant following the exploit method
        if(norm_random()>=random_rate):
            allowed_nodes = []
            sum_of_probs = 0.0
            #get all possible nodes
            for i in range(len(data)):
                if not (i in self.path): 
                    allowed_nodes.append(i)
                    sum_of_probs += attract_pherm(x= self.path[-1] , y= i , alpha=alpha,beta=beta,pheromone=pheromone,data=data)
            #counting probabilities
            max_prob = 0

            for node in allowed_nodes:
                prob_xy = (attract_pherm(x= self.path[-1] , y= node , alpha=alpha,beta=beta,pheromone=pheromone,data=data))/sum_of_probs
             
                if prob_xy >= max_prob : 
                    next_node = node
                    max_prob = prob_xy
           
        #case of an ant following the exploring method
        else : 
            node = 0 
            while(node in self.path):
                node +=1
            next_node = node

        #adding cost 
        self.cost+= data[self.path[-1]][next_node]

        if(len(self.path)==len(data)):
            self.cost+= data[self.path[-1]][0]
            
        self.path.append(next_node)
    
    #updating pheromone 
    def update_pheromone(self, pheromone,q):
        for i in range(len(self.path)-1):
            pheromone[self.path[i]][self.path[i+1]]+= q/self.cost
            pheromone[self.path[i+1]][self.path[i]]+= q/self.cost
        new_ph = pheromone
        return new_ph
    
    #reset path
    def reset_path(self):
        self.cost = 0.0
        self.path = [self.path[0]]

## Initializing procedures

In [29]:
def initialize_pheromone_matrix(data):
    m = np.ones(data.shape)
    for i in range(len(m)):
        m[i][i]= 0
    return m 

In [30]:
def initialize_colony(ants_number, data):
    ants= []
    begin = np.random.randint(len(data))
    for i in range(ants_number):
        ants.append(Ant(begin))
    return ants , begin

## Ant Colony Algorithm 

In [31]:
def AntColony(data,ants_number = 4, alpha = 0.7 , beta = 0.4 , evaporate_rate = 0.1,random_rate=0.5 , show_history = False):
    assert ants_number >=2 
    path = []
    cost = 0.0
    stop = False 
    pheromone = initialize_pheromone_matrix(data)
    ants , begin = initialize_colony(ants_number=ants_number, data=data)
    while (not stop) : 
        #first : visit nodes with ants
        for ant in ants:
            #looking for a cycle
            while (len(ant.path) != len(data)):
                ant.next_node(data=data,pheromone=pheromone, alpha=alpha, beta = beta,random_rate=random_rate)
        
        #second : evaporation 
        pheromone=np.dot(pheromone,1-evaporate_rate)
        
        #last : update the pheremone in nodes 
        for ant in ants:
            pheromone = ant.update_pheromone(pheromone=pheromone,q=10)
        
        #checking the stop condition
        path = ants[0].path
        stop = True
        for ant in ants[1:]:
            stop = stop and np.array_equal(path,ant.path)
        random_rate = random_rate / 10
        
        #an optional feature to print the history of paths
        if show_history :  
            show_paths(ants=ants)

        #reseting paths 
        for ant in ants :
            if not stop : ant.reset_path()
    else :
        return ants[0].path , ants[0].cost

In [33]:
p,c = AntColony(data=data,ants_number=4,alpha = 0.5,beta = 0.1,evaporate_rate=0.1,random_rate=0.7,show_history=False)
print(p , c)

[1, 56, 86, 96, 94, 93, 5, 95, 98, 58, 91, 97, 36, 99, 90, 84, 92, 60, 15, 85, 43, 13, 41, 42, 14, 40, 21, 73, 71, 72, 20, 39, 57, 52, 100, 26, 68, 0, 49, 75, 76, 2, 78, 80, 32, 50, 8, 70, 34, 33, 77, 28, 23, 79, 67, 11, 25, 27, 88, 51, 17, 82, 59, 4, 83, 16, 44, 7, 81, 47, 46, 35, 48, 18, 10, 62, 89, 31, 29, 69, 30, 87, 6, 61, 9, 19, 65, 64, 63, 45, 12, 74, 55, 3, 54, 24, 38, 22, 66, 53, 37] 797.0


## Running algorithm multiple times to get lower cost

In [38]:
p = [] 
c = np.inf
for i in range(20): 
    path , cost = AntColony(data=data,ants_number=5,alpha = 0.5,beta = 0.1,evaporate_rate=0.1,random_rate=0.7,show_history=False)
    if(cost < c ):
        c = cost 
        p = path

In [39]:
print(p , c)

[97, 36, 99, 90, 84, 92, 98, 95, 58, 91, 96, 94, 93, 5, 88, 51, 17, 82, 59, 4, 83, 16, 44, 7, 81, 47, 46, 35, 48, 63, 62, 89, 31, 29, 69, 30, 87, 6, 61, 9, 10, 18, 45, 60, 15, 43, 13, 41, 86, 56, 1, 72, 71, 73, 21, 40, 74, 55, 3, 53, 54, 24, 38, 22, 66, 20, 39, 57, 52, 100, 26, 68, 0, 49, 75, 76, 2, 78, 77, 33, 34, 70, 8, 80, 32, 50, 19, 65, 64, 28, 23, 79, 67, 11, 25, 27, 12, 14, 42, 37, 85] 728.0
